In [12]:
import pandas as pd
from scipy import stats
import numpy as np

In [13]:
df = pd.read_csv(r"f:\Paper\ResearchData\TxnPrediction Data\Processed Data\finalData.csv")
df = df.drop(df.columns[0], axis=1)

C:\Users\Umesh Bhatt\AppData\Local\Temp\ipykernel_2108\1956207060.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"f:\Paper\ResearchData\TxnPrediction Data\Processed Data\finalData.csv")


In [14]:
df.head()



,blockNumber,transactionIndex,gasUsed,gasPrice,maxFeePerGas,maxPriorityFeePerGas,baseFees,value,transactionFee,priorityFee,timstamp,voteCount,activeValidators,slot,epoch
0,18000001,0,21000.0,5.550000e+11,555000000000,555000000000,4.608838e+14,80000000000000000,1.165500e+16,1.119412e+16,1693066907,23471,753577,7186907,224590
1,18000001,1,247293.0,9.694685e+10,107581637462,75000000000,5.427302e+15,50000000000000000,2.397428e+16,1.854698e+16,1693066907,23471,753577,7186907,224590
2,18000001,2,264383.0,7.294685e+10,83581637462,51000000000,5.802374e+15,100000000000000000,1.928591e+16,1.348353e+16,1693066907,23471,753577,7186907,224590
3,18000001,3,267804.0,2.194685e+10,21946848177,0,5.877454e+15,30826777,5.877454e+15,0.000000e+00,1693066907,23471,753577,7186907,224590
4,18000001,4,196123.0,2.204685e+10,28694459494,100000000,4.304282e+15,500000000000000000,4.323894e+15,1.961230e+13,1693066907,23471,753577,7186907,224590


In [15]:
df = df.drop(columns=['blockNumber', 'transactionIndex', 'priorityFee', 'slot', 'epoch'], axis=1)
df.head()
numeric_columns = ['maxFeePerGas', 'maxPriorityFeePerGas', 'value','gasUsed', 'gasPrice',  'transactionFee', 'baseFees',"voteCount", "activeValidators",'timstamp']
df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')
df.shape

(2682683, 10)

In [16]:
df = df[(df != -1).all(axis=1)]


In [17]:
# Convert columns from wei to Gwei
wei_to_Gwei = 10**9
df['gasPrice'] /= wei_to_Gwei
df['gasUsed'] /= wei_to_Gwei
df['maxFeePerGas'] /= wei_to_Gwei
df['maxPriorityFeePerGas'] /= wei_to_Gwei
df['baseFees'] /= wei_to_Gwei
df['value'] /= wei_to_Gwei
df['transactionFee'] /= wei_to_Gwei

df.head()

,gasUsed,gasPrice,maxFeePerGas,maxPriorityFeePerGas,baseFees,value,transactionFee,timstamp,voteCount,activeValidators
0,0.000021,555.000000,555.000000,555.0,4.608838e+05,8.000000e+07,1.165500e+07,1693066907,23471,753577
1,0.000247,96.946848,107.581637,75.0,5.427302e+06,5.000000e+07,2.397428e+07,1693066907,23471,753577
2,0.000264,72.946848,83.581637,51.0,5.802374e+06,1.000000e+08,1.928591e+07,1693066907,23471,753577
3,0.000268,21.946848,21.946848,0.0,5.877454e+06,3.082678e-02,5.877454e+06,1693066907,23471,753577
4,0.000196,22.046848,28.694459,0.1,4.304282e+06,5.000000e+08,4.323894e+06,1693066907,23471,753577


In [18]:
z_scores = np.abs(stats.zscore(df))
outliers = (z_scores > 3).any(axis=1)

# Remove outliers
df = df[~outliers]
df.head()

multiplier = 1.5
df_no_outliers = df.copy()

for column in df.columns:
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - multiplier * IQR
    upper_bound = Q3 + multiplier * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    df_no_outliers = df_no_outliers[~df_no_outliers.index.isin(outliers.index)]
df = df_no_outliers
df.head()

,gasUsed,gasPrice,maxFeePerGas,maxPriorityFeePerGas,baseFees,value,transactionFee,timstamp,voteCount,activeValidators
3,0.000268,21.946848,21.946848,0.0,5.877454e+06,0.030827,5.877454e+06,1693066907,23471,753577
15,0.000248,24.946848,35.581637,3.0,5.438297e+06,0.000000,6.181679e+06,1693066907,23471,753577
16,0.000264,24.946848,35.581637,3.0,5.792914e+06,0.000000,6.584770e+06,1693066907,23471,753577
19,0.000169,23.946848,30.000000,2.0,3.714745e+06,0.000000,4.053267e+06,1693066907,23471,753577
20,0.000196,23.946848,30.000000,2.0,4.293023e+06,0.000000,4.684243e+06,1693066907,23471,753577


In [19]:
# df_500K = df.iloc[:500000]
# df_200K = df.iloc[:200000]
# df_100K = df.iloc[:100000]

# print(df_500K.shape)
# print(df_200K.shape)
# print(df_100K.shape)

In [20]:
# df_500K.to_csv("TxTimeData_500K.csv")
# df_200K.to_csv("TxTimeData_200K.csv")
# df_100K.to_csv("TxTimeData_100K.csv")